In [1]:
import geopandas as gpd
from imgw_static import get_map_zlewnie, get_static_data_hydro
from imgw_api import get_hydro_metadata, get_meteo_metadata
import pandas as pd
import numpy as np

# Preprocessing

## Dane Hydrologiczne

Dane Pomiarowe

In [4]:
df = get_static_data_hydro()

In [5]:
print("Shape: ", df.shape)
print("Ilość stacji hydro w danych pomiarowych: ",df["Station Code"].nunique())
df.head()


Shape:  (4273340, 10)
Ilość stacji hydro w danych pomiarowych:  917


,Station Code,Station Name,Name,Hydro Year,Hydro Month,Day,Water Level,Flow,Water Temp,Calendar Month
0,149180020,CHA£UPKI,Odra (1),2010,1,1,158,30.8,99.9,11
1,149180020,CHA£UPKI,Odra (1),2010,1,2,154,28.5,99.9,11
2,149180020,CHA£UPKI,Odra (1),2010,1,3,150,26.0,99.9,11
3,149180020,CHA£UPKI,Odra (1),2010,1,4,149,25.4,99.9,11
4,149180020,CHA£UPKI,Odra (1),2010,1,5,149,25.9,99.9,11


Dane Meta

In [7]:
gdf = get_hydro_metadata()
print("Shape: ", gdf.shape)
print("Ilość stacji hydro w meta danych: ",gdf["Station Code"].nunique())
gdf.head()

Shape:  (860, 5)
Ilość stacji hydro w meta danych:  860


,Station Code,Station Name,Lon,Lat,geometry
0,150160330,SZCZYTNA,16.443056,50.415556,POINT (318401.478 286283.506)
1,150160340,SARNY,16.465833,50.547778,POINT (320520.661 300923.221)
2,150160350,SZALEJÓW GÓRNY,16.537222,50.418333,POINT (325098.752 286366.241)
3,150160360,STARKÓW,16.580000,50.377500,POINT (327988.637 281728.229)
4,150160370,TOPOLICE,16.609167,50.366944,POINT (330023.848 280487.958)


## Wstępna selekcja danych hydrologicznych

Wybieramy stacje które:

1. Rozpoczęły pomiary przed lub w 2010 roku.
2. Można zmapować do ich lokalizacji.

In [9]:
df['Calendar Year'] = np.where(df['Hydro Month'] <3, df['Hydro Year']-1, df['Hydro Year'])
df = df.loc[df['Calendar Year'] < 2023, :]
df = df.loc[df['Calendar Year'] >= 2010, :]
df

,Station Code,Station Name,Name,Hydro Year,Hydro Month,Day,Water Level,Flow,Water Temp,Calendar Month,Calendar Year
0,149180020,CHA£UPKI,Odra (1),2010,3,1,201,71.90,99.9,1,2010
1,149180020,CHA£UPKI,Odra (1),2010,3,2,202,73.00,99.9,1,2010
2,149180020,CHA£UPKI,Odra (1),2010,3,3,193,64.10,99.9,1,2010
3,149180020,CHA£UPKI,Odra (1),2010,3,4,181,51.70,99.9,1,2010
4,149180020,CHA£UPKI,Odra (1),2010,3,5,180,51.40,99.9,1,2010
...,...,...,...,...,...,...,...,...,...,...,...
301685,149190250,JABŁONKA,Piekielnik (82224),2023,2,27,161,3.08,99.9,12,2022
301686,149190250,JABŁONKA,Piekielnik (82224),2023,2,28,157,2.42,99.9,12,2022
301687,149190250,JABŁONKA,Piekielnik (82224),2023,2,29,153,1.80,99.9,12,2022
301688,149190250,JABŁONKA,Piekielnik (82224),2023,2,30,151,1.32,99.9,12,2022


In [10]:
df['Calendar Date'] = df['Calendar Year'].astype(str) + '-' + \
                        df['Calendar Month'].astype(str).str.zfill(2) + '-' + \
                        df['Day'].astype(str).str.zfill(2)
df['Calendar Date'] = pd.to_datetime(df['Calendar Date'])

Odcinamy stacje z pierszwego warunku.

In [12]:
valid_stations = df.groupby("Station Code")["Calendar Year"].min()
valid_stations = valid_stations[valid_stations <= 2010]

filtered_df = df[df["Station Code"].isin(valid_stations.index)]

In [13]:
filtered_df['Station Code'].nunique()

866

Odcinamy stacje z drugiego warunku.

In [15]:
valid_stations = gdf['Station Code']

filtered_df_2 = filtered_df[filtered_df["Station Code"].isin(valid_stations)]

In [16]:
print("Shape: ", filtered_df_2.shape)
print("Ilość stacji w odfiltrowanych hydro danych: ",filtered_df_2["Station Code"].nunique())

Shape:  (3376666, 12)
Ilość stacji w odfiltrowanych hydro danych:  719


### Mapowanie Kodów 

Stan wody 9999 oznacza brak danych w bazie.

Przepływ 99999.999 oznacza, że przepływ w tym dniu nie był opracowywany.

Temperatura 99.9 oznacza brak danych w bazie, która może wynikać np. z braku pomiarów temperatury na stacji.

In [18]:
filtered_df_2.replace({
    'Water Level': {9999: np.nan},
    'Flow': {99999.999: np.nan},
    'Water Temp': {99.9: np.nan}
}, inplace=True)

/var/folders/z9/f473nv411j13cywwsnc_vt580000gn/T/ipykernel_45035/121063041.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  filtered_df_2.replace({


Eksportujemy odfiltrowane dane do static_data.

W efekcie mamy spójne dane zaczynające się od 2010 i z lokalizacją i z zmapowanymi wartościami brakującymi.

In [20]:
filtered_df_2.to_parquet("../static_data/filtered_hydro_data.parquet.gzip")

## Dane Meteorologiczne Opadowe

Dane pomiarowe

In [23]:
df = pd.read_parquet('../static_data/meteo_opad_data.parquet.gzip')

In [24]:
print("Shape: ", df.shape)
print("Ilość stacji meteo opadowych w danych pomiarowych: ",df["Station Code"].nunique())

Shape:  (1801634, 16)
Ilość stacji meteo opadowych w danych pomiarowych:  1137


Metadane

In [26]:
gdf = get_meteo_metadata()
print("Shape: ", gdf.shape)
print("Ilość stacji meteo w meta danych: ",gdf["Station Code"].nunique())
gdf.head()

Shape:  (766, 5)
Ilość stacji meteo w meta danych:  766


,Station Code,Station Name,Lon,Lat,geometry
0,252210290,RYBIENKO,21.429167,52.577778,POINT (664545.901 526326.119)
1,351230497,WŁODAWA,23.529444,51.553333,POINT (813865.629 419371.773)
2,352220385,SIEDLCE,22.244722,52.181111,POINT (721755.946 484409.713)
3,250240010,STRZYŻÓW,24.035556,50.840000,POINT (854357.945 342430.384)
4,352230399,TERESPOL,23.621944,52.078611,POINT (816563.802 478131.955)


## Wstępna selekcja danych meteorologicznych

Wybieramy stacje które:

1. Rozpoczęły pomiary przed lub w 2010 roku.
2. Można zmapować do ich lokalizacji.

In [28]:
valid_stations = df.groupby("Station Code")["Year"].min()
valid_stations = valid_stations[valid_stations <= 2010].index

filtered_df = df[df["Station Code"].isin(valid_stations)]

In [29]:
print("Ilość stacji w odfiltrowanych (filtracja czasowa) meteo danych: ",filtered_df["Station Code"].nunique())

Ilość stacji w odfiltrowanych (filtracja czasowa) meteo danych:  958


In [30]:
valid_stations = gdf['Station Code']

filtered_df_2 = filtered_df[filtered_df["Station Code"].isin(valid_stations)]

In [31]:
print("Shape: ", filtered_df_2.shape)
print("Ilość stacji w odfiltrowanych meteo danych: ",filtered_df_2["Station Code"].nunique())

Shape:  (605780, 16)
Ilość stacji w odfiltrowanych meteo danych:  274


In [32]:
date_str = filtered_df_2['Year'].astype(str) + '-' + \
                        filtered_df_2['Month'].astype(str).str.zfill(2) + '-' + \
                        filtered_df_2['Day'].astype(str).str.zfill(2)
filtered_df_2['Calendar Date'] = pd.to_datetime(date_str, errors='coerce')

/var/folders/z9/f473nv411j13cywwsnc_vt580000gn/T/ipykernel_45035/3497555919.py:4: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  filtered_df_2['Calendar Date'] = pd.to_datetime(date_str, errors='coerce')


### Weryfikacja statusów

Oznaczaenia w kolumnach status:
- 8 - brak pomiaru 
- 9 - brak zjawiska

In [35]:
filtered_df_2.loc[filtered_df_2['SMDB Measurement Status']==8, 'Daily Precip Sum'].unique()

array([0.])

8 w kolumnie ze statusem pomiaru opadu oznacza brak pomiaru, a w kolumnie z sumą opadów są 0, co sugeruje brak zjawiska.
Zamianiamy 0 w kolumnach z pomiarami na Nan, gdy według kolumn ze statusami, nie wykonano pomairu.

In [37]:
filtered_df_2.loc[filtered_df_2['SMDB Measurement Status']== 8,'Daily Precip Sum'] = np.nan
filtered_df_2.loc[filtered_df_2['PKSN Measurement Status']== 8,'Snow Cover Height'] = np.nan
filtered_df_2.loc[filtered_df_2['HSS Measurement Status']== 8,'Fresh Snow Height'] = np.nan

Ze względu na brak zastosowania usuwamy kolumny z infomracjami o gatunku śniegu i rodzaj pokrywy śnieżej.
Usuwamy także kolmy ze statusami, ponieważ te same informacje zostały już zawarte w kolumnach z wartościami pomiarów.

In [39]:
filtered_df_2.drop(columns = ['SMDB Measurement Status', 'PKSN Measurement Status', 
       'HSS Measurement Status', 'Snow Type', 'GATS Measurement Status',
       'Snow Cover Type', 'RPSN Measurement Status'], inplace = True)

/var/folders/z9/f473nv411j13cywwsnc_vt580000gn/T/ipykernel_45035/885459402.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  filtered_df_2.drop(columns = ['SMDB Measurement Status', 'PKSN Measurement Status',


Eksportujemy odfiltrowane dane do static_data.

In [41]:
filtered_df_2.to_parquet("../static_data/filtered_meteo_opad_data.parquet.gzip")